# Clusters
The aim of this section is cluster the crime density in Ottawa and explore if the center of these clusters are close to the real positions of the services in Ottawa.

## Load Clean data

In [1]:
import pandas as pd
import altair as alt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go 
from sklearn.cluster import DBSCAN

In [ ]:
neighbourhood_loc = pd.read_pickle('../clean_datasets/Neighbourhood.pkl')
police_centers = pd.read_pickle("../clean_datasets/police_centers.pkl")
criminal_data_set = pd.read_csv('../datasets/Criminal_Offences_Open_Data.csv')
hospitals = pd.read_pickle("../clean_datasets/hospitals.pkl")
hospitals

C:\Users\Owner\AppData\Local\Temp\ipykernel_31292\2799728222.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  criminal_data_set = pd.read_csv('../datasets/Criminal_Offences_Open_Data.csv')


,OBJECTID,ID,NAME,ADDRESS,PHONE,LINK_LABEL_EN,LINK_EN,LINK_DESCRIPTION_EN,LINK_LABEL_FR,LINK_FR,GLOBALID,ADDRESS_FR,NAME_FR,geometry,lat,long
0,1,1,Royal Ottawa Hospital,1145 Carling Avenue,722-6521,NaN,http://www.theroyal.ca/,NaN,NaN,http://www.leroyal.ca/,{E279DD71-C8D3-4C76-B317-1ACF4DB9281A},"1145, avenue Carling",Hôpital Royal Ottawa,POINT Z (-75.73236 45.38809 0.00000),45.388086,-75.732364
1,2,2,Ottawa Hospital - Civic Campus,1053 Carling Avenue,761-4000,NaN,https://www.ottawahospital.on.ca/,NaN,NaN,https://www.ottawahospital.on.ca/,{C1661EEA-B6C5-4C11-9789-E5353940777A},"1053, avenue Carling",Hôpital d’Ottawa – Campus Civic,POINT Z (-75.72121 45.39267 0.00000),45.392671,-75.721212
2,3,3,Ottawa Hospital - Riverside Campus,1967 Riverside Drive,738-7100,NaN,https://www.ottawahospital.on.ca/,NaN,NaN,https://www.ottawahospital.on.ca/,{7A30AEC4-D042-41D6-9CF1-D06F6202137F},"1967, promenade Riverside","Hôpital d’Ottawa, campus Riverside",POINT Z (-75.66794 45.39675 0.00000),45.396750,-75.667941
3,4,4,Children's Hospital of Eastern Ontario,401 Smyth Road,737-7600,NaN,http://www.cheo.on.ca/,NaN,NaN,http://www.cheo.on.ca/fr/home,{93E3EACA-FB11-49E9-A914-0D5E8B5CBF7B},"401, chemin Smyth",Centre hospitalier pour enfants de l'est de l'Ont,POINT Z (-75.65192 45.40102 0.00000),45.401019,-75.651923
4,5,5,Ottawa Hospital - General Campus,501 Smyth Road,737-7777,NaN,https://www.ottawahospital.on.ca/,NaN,NaN,https://www.ottawahospital.on.ca/,{CC861544-A63C-4A64-89DA-65BC7274827E},"501, chemin Smyth",Hôpital d’Ottawa – Campus Général,POINT Z (-75.64764 45.40099 0.00000),45.400987,-75.647642
5,6,6,Montfort Hospital,713 Montreal Road,746-4621,NaN,http://www.hopitalmontfort.com/en,NaN,NaN,http://www.hopitalmontfort.com/,{D394D8AA-B8B6-4C7E-B4A8-80A37F4B8536},"713, chemin Montréal",Hôpital Montfort,POINT Z (-75.63879 45.44588 0.00000),45.445883,-75.638787
6,7,7,Queensway-Carleton Hospital,3045 Baseline Road,721-2000,NaN,http://www.qch.on.ca/,NaN,NaN,http://www.qch.on.ca/,{AF1DD3FB-C781-4D7F-ADFA-C2E8B7CEF974},"3045, chemin Baseline",Hôpital Queensway-Carleton,POINT Z (-75.80755 45.33498 0.00000),45.334976,-75.807550
7,8,8,Rehabilitation Centre,505 Smyth Road,737-7350,NaN,https://www.ottawahospital.on.ca/,NaN,NaN,https://www.ottawahospital.on.ca/,{97CA4D29-4EBB-43C1-ADC4-C8FC087DCCF0},"505, chemin Smyth",Centre de réadaptation,POINT Z (-75.65180 45.40422 0.00000),45.404216,-75.651798
8,9,9,Saint Vincent Hospital,60 Cambridge Street North,233-4041,NaN,https://www.bruyere.org/en/home,NaN,NaN,https://www.bruyere.org/fr/home,{9AEE6E9C-005C-4954-A59B-8DF73E494228},"60, rue Cambridge Nord",Hôpital Saint-Vincent,POINT Z (-75.70897 45.41385 0.00000),45.413854,-75.708969
9,10,10,Elizabeth Bruyere Hospital,43 Bruyere Street,562-0050,NaN,http://www.bruyere.org/en/elisabeth-bruyere-ho...,NaN,NaN,http://www.bruyere.org/fr/programmes-services-...,{A306EA35-1B58-424A-A2E6-167EB2801297},"43, rue Bruyère",Hôpital de jour Elizabeth Bruyère,POINT Z (-75.69748 45.43191 0.00000),45.431905,-75.697481


## Police centers

In [10]:
scaler = StandardScaler()
scaler.fit(criminal_data_set[["X", "Y"]])
kmeans = KMeans(n_clusters=10, random_state=0, n_init="auto").fit(scaler.transform(criminal_data_set[["X", "Y"]]))

In [11]:
police_centers_clusters = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns =["X", "Y"])
police_centers_clusters.to_pickle("../clusters/police_clusters.pkl")
police_centers_clusters

,X,Y
0,-75.646659,45.363104
1,-75.908493,45.295461
2,-75.778347,45.347432
3,-75.631784,45.428082
4,-75.736362,45.267597
5,-75.495025,45.460214
6,-75.602359,45.204390
7,-75.733770,45.381434
8,-75.690099,45.420261
9,-76.125305,45.448044


In [12]:
alt.Chart(criminal_data_set[["X", "Y"]][:5000]).mark_circle().encode(
    x =alt.X("X").scale(zero=False),
    y = alt.X("Y").scale(zero=False)
)+alt.Chart(pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns =["X", "Y"])).mark_circle(size=100, 
                                        color='red').encode(
    x =alt.X("X"),
    y = alt.X("Y")
)

alt.LayerChart(...)

## Hospitals

In [13]:
## Select crimes that ar related to humain harm
crime_against_person = criminal_data_set[criminal_data_set["OFF_CATEG"].isin(["Assaults",
                                                       'Other Violations Involving Violence Or The Threat Of Violence',
                                                       "Violations Causing Death",
                                                       "Arson",
                                                       "Failure to Stop after Accident", ])].reset_index()

### Using kmeans

In [14]:
scaler = StandardScaler()
scaler.fit(crime_against_person[["X", "Y"]])
kmeans = KMeans(n_clusters=10, random_state=0, n_init="auto").fit(scaler.transform(crime_against_person[["X", "Y"]]))
hospitals_clusters = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns =["X", "Y"])
hospitals_clusters.to_pickle("../clusters/hospitals_kmeans.pkl")

### Using DBscan then kmeans 

In [15]:
dbscan = DBSCAN(eps=0.0001, min_samples= 50).fit(crime_against_person[["X", "Y"]])
db_label = pd.DataFrame(dbscan.labels_, columns = ["label"])
db_data = pd.concat([crime_against_person, db_label], axis = 1)
db_data = db_data[db_data.label != -1]

In [16]:
scaler = StandardScaler()
scaler.fit(db_data[["X","Y"]])
kmeans = KMeans(n_clusters=10, random_state=0, n_init="auto").fit(scaler.transform(db_data[["X","Y"]]))
dbscan_clusters = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns =["X", "Y"])
dbscan_clusters.to_pickle("../clusters/hospitals_dbscan.pkl")